In [59]:
from langchain_core.tools import tool
from langchain_groq import ChatGroq
from langchain.messages import HumanMessage,AIMessage
from langchain_core.tools import InjectedToolArg
from typing import Annotated
import json
import os
from dotenv import load_dotenv
load_dotenv()
groq_api_key=os.getenv("GROQ_API_KEY")
exchange_api = os.getenv("EXCHANGE_RATE_API")
import requests

In [3]:
@tool
def multiply(a :int, b:int)->int:
    """Given two numbers a and b this tool returns their multiplication
    """
    return a*b


In [4]:
llm = ChatGroq(groq_api_key=groq_api_key,model_name="llama-3.1-8b-instant")

In [5]:
llm_with_tools = llm.bind_tools([multiply])

In [23]:
query = HumanMessage("can you multiply 3 and 4")

messages = [query]
print(messages)

[HumanMessage(content='can you multiply 3 and 4', additional_kwargs={}, response_metadata={})]


In [25]:
result = (llm_with_tools.invoke("can you multiply 3 and 4"))
messages.append(result)

In [26]:
print(messages)

[HumanMessage(content='can you multiply 3 and 4', additional_kwargs={}, response_metadata={}), AIMessage(content='', additional_kwargs={'tool_calls': [{'id': '62gqwrwxq', 'function': {'arguments': '{"a":3,"b":4}', 'name': 'multiply'}, 'type': 'function'}]}, response_metadata={'token_usage': {'completion_tokens': 19, 'prompt_tokens': 236, 'total_tokens': 255, 'completion_time': 0.037921216, 'completion_tokens_details': None, 'prompt_time': 0.022782843, 'prompt_tokens_details': None, 'queue_time': 0.053850426, 'total_time': 0.060704059}, 'model_name': 'llama-3.1-8b-instant', 'system_fingerprint': 'fp_4387d3edbb', 'service_tier': 'on_demand', 'finish_reason': 'tool_calls', 'logprobs': None, 'model_provider': 'groq'}, id='lc_run--019bd222-aa56-7b92-ab69-697a9ab88bbf-0', tool_calls=[{'name': 'multiply', 'args': {'a': 3, 'b': 4}, 'id': '62gqwrwxq', 'type': 'tool_call'}], usage_metadata={'input_tokens': 236, 'output_tokens': 19, 'total_tokens': 255})]


In [27]:
tool_Result = multiply.invoke(result.tool_calls[0])
messages.append(tool_Result)

In [28]:
print(messages)

[HumanMessage(content='can you multiply 3 and 4', additional_kwargs={}, response_metadata={}), AIMessage(content='', additional_kwargs={'tool_calls': [{'id': '62gqwrwxq', 'function': {'arguments': '{"a":3,"b":4}', 'name': 'multiply'}, 'type': 'function'}]}, response_metadata={'token_usage': {'completion_tokens': 19, 'prompt_tokens': 236, 'total_tokens': 255, 'completion_time': 0.037921216, 'completion_tokens_details': None, 'prompt_time': 0.022782843, 'prompt_tokens_details': None, 'queue_time': 0.053850426, 'total_time': 0.060704059}, 'model_name': 'llama-3.1-8b-instant', 'system_fingerprint': 'fp_4387d3edbb', 'service_tier': 'on_demand', 'finish_reason': 'tool_calls', 'logprobs': None, 'model_provider': 'groq'}, id='lc_run--019bd222-aa56-7b92-ab69-697a9ab88bbf-0', tool_calls=[{'name': 'multiply', 'args': {'a': 3, 'b': 4}, 'id': '62gqwrwxq', 'type': 'tool_call'}], usage_metadata={'input_tokens': 236, 'output_tokens': 19, 'total_tokens': 255}), ToolMessage(content='12', name='multiply'

In [29]:
llm_with_tools.invoke(messages)

AIMessage(content='The result of the multiplication is 12.', additional_kwargs={}, response_metadata={'token_usage': {'completion_tokens': 10, 'prompt_tokens': 266, 'total_tokens': 276, 'completion_time': 0.019062332, 'completion_tokens_details': None, 'prompt_time': 0.023505575, 'prompt_tokens_details': None, 'queue_time': 0.086428005, 'total_time': 0.042567907}, 'model_name': 'llama-3.1-8b-instant', 'system_fingerprint': 'fp_4387d3edbb', 'service_tier': 'on_demand', 'finish_reason': 'stop', 'logprobs': None, 'model_provider': 'groq'}, id='lc_run--019bd222-ab93-79e2-8244-3a7370d45082-0', usage_metadata={'input_tokens': 266, 'output_tokens': 10, 'total_tokens': 276})

MULTI TOOL IMPLEMENTATION

In [66]:
@tool
def currency_converter(base_currency:str,target_currency:str)->float:
    """This tool gives the conversion factor between a base currency and a target currency"""

    url = f'https://v6.exchangerate-api.com/v6/{exchange_api}/pair/{base_currency}/{target_currency}'
    response = requests.get(url)
    return response.json()


@tool
def convert(base_currency_value:int, conversion_factor:Annotated[float,InjectedToolArg])->float:
    """Given a conversion factor this function converts base currency value into target currency value """
    return base_currency_value*conversion_factor


In [67]:
c_factor = currency_converter.invoke({"base_currency":"USD","target_currency":"INR"})
print(c_factor)

{'result': 'success', 'documentation': 'https://www.exchangerate-api.com/docs', 'terms_of_use': 'https://www.exchangerate-api.com/terms', 'time_last_update_unix': 1768694401, 'time_last_update_utc': 'Sun, 18 Jan 2026 00:00:01 +0000', 'time_next_update_unix': 1768780801, 'time_next_update_utc': 'Mon, 19 Jan 2026 00:00:01 +0000', 'base_code': 'USD', 'target_code': 'INR', 'conversion_rate': 90.7742}


In [68]:
llm_with_tools2= llm.bind_tools([convert,currency_converter])

In [69]:
query=HumanMessage("whta is conversion rate between USD and INR and based on that can you convert 10 usd to inr")
messages2=[query]

In [70]:
ai_message=llm_with_tools2.invoke(messages2)
messages2.append(ai_message)

In [71]:
print(ai_message.tool_calls)

[{'name': 'currency_converter', 'args': {'base_currency': 'USD', 'target_currency': 'INR'}, 'id': '0fmcfgp3y', 'type': 'tool_call'}, {'name': 'convert', 'args': {'base_currency_value': 10}, 'id': '25h7dffek', 'type': 'tool_call'}]


In [72]:

for tool_call in ai_message.tool_calls:
    if tool_call['name']=='currency_converter':
        tool_message1= currency_converter.invoke(tool_call)
        conversion_rate = json.loads(tool_message1.content)['conversion_rate']
        messages2.append(tool_message1)
    
    if tool_call['name'] == 'convert' :
        tool_call['args']['conversion_factor'] = conversion_rate
        tool_message2 = convert.invoke(tool_call)
        messages2.append(tool_message2)


In [73]:
print(messages2)

[HumanMessage(content='whta is conversion rate between USD and INR and based on that can you convert 10 usd to inr', additional_kwargs={}, response_metadata={}), AIMessage(content='', additional_kwargs={'tool_calls': [{'id': '0fmcfgp3y', 'function': {'arguments': '{"base_currency":"USD","target_currency":"INR"}', 'name': 'currency_converter'}, 'type': 'function'}, {'id': '25h7dffek', 'function': {'arguments': '{"base_currency_value":10}', 'name': 'convert'}, 'type': 'function'}]}, response_metadata={'token_usage': {'completion_tokens': 38, 'prompt_tokens': 333, 'total_tokens': 371, 'completion_time': 0.062224502, 'completion_tokens_details': None, 'prompt_time': 0.02692188, 'prompt_tokens_details': None, 'queue_time': 0.05316319, 'total_time': 0.089146382}, 'model_name': 'llama-3.1-8b-instant', 'system_fingerprint': 'fp_ff2b098aaf', 'service_tier': 'on_demand', 'finish_reason': 'tool_calls', 'logprobs': None, 'model_provider': 'groq'}, id='lc_run--019bd276-42ec-7023-86a9-a136da46d97c-0

In [74]:
llm_with_tools2.invoke(messages2)

AIMessage(content='The conversion rate between USD and INR is 90.7742. Therefore, 10 USD is equivalent to 907.742 INR.', additional_kwargs={}, response_metadata={'token_usage': {'completion_tokens': 31, 'prompt_tokens': 535, 'total_tokens': 566, 'completion_time': 0.044031329, 'completion_tokens_details': None, 'prompt_time': 0.23061758, 'prompt_tokens_details': None, 'queue_time': 0.072726299, 'total_time': 0.274648909}, 'model_name': 'llama-3.1-8b-instant', 'system_fingerprint': 'fp_f757f4b0bf', 'service_tier': 'on_demand', 'finish_reason': 'stop', 'logprobs': None, 'model_provider': 'groq'}, id='lc_run--019bd277-13ac-72f1-97c0-16787307ecbd-0', usage_metadata={'input_tokens': 535, 'output_tokens': 31, 'total_tokens': 566})